<a href="https://colab.research.google.com/github/gtanvi58/mass-shooter-classifier/blob/master/BERT_models_with_CL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Go to Runtime --> change runtime type and select TPU.
Then, run the below commands, and restart the kernal to reflect changes.


In [ ]:
!pip install accelerate -U
!pip install transformers

Set seed

In [ ]:
import torch

# Set a random seed for CPU
torch.manual_seed(42)

# Set a random seed for GPU
if torch.cuda.is_available():
    torch.cuda.manual_seed(42)
    torch.cuda.manual_seed_all(42)

# Other random initializations
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False


BERT

InfoNCE

In [ ]:
import pandas as pd
import json
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
import torch
from transformers import TrainingArguments, Trainer
from transformers import BertTokenizer, BertForSequenceClassification
from torch.nn import CrossEntropyLoss
import torch.nn.functional as F

# Load data from JSON files
with open('./annotated_data-train.json', 'r') as f:
    train_data = json.load(f)

with open('./annotated_data-test.json', 'r') as f:
    test_data = json.load(f)

# Convert data to Pandas DataFrame
train_df = pd.DataFrame(train_data)
test_df = pd.DataFrame(test_data)

# Concatenate train and test dataframes
df = pd.concat([train_df, test_df])

# Select 'text' and 'label' columns
df = df[['text', 'label']]

# Map label values to integers
label_map = {'noHate': 0, 'hate': 1}
df['label_value'] = df['label'].map(label_map)

# Reset index
df.reset_index(drop=True, inplace=True)

# Select 'text' and 'label_value' columns
data = df[['text', 'label_value']]

# Initialize BERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

# Move model to GPU
model = model.to('cuda')

# Split data into train and validation sets
X = list(data["text"])
y = list(data["label_value"])
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, stratify=y)

# Tokenize train and validation sets
X_train_tokenized = tokenizer(X_train, padding=True, truncation=True, max_length=512)
X_val_tokenized = tokenizer(X_val, padding=True, truncation=True, max_length=512)

# Define Dataset class
class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

# Create train and validation datasets
train_dataset = Dataset(X_train_tokenized, y_train)
val_dataset = Dataset(X_val_tokenized, y_val)

# Define InfoNCE loss function
class InfoNCELoss(torch.nn.Module):
    def __init__(self):
        super(InfoNCELoss, self).__init__()

    def forward(self, outputs, labels):
        # Retrieve the projected and predicted representations
        projected_representation = outputs.logits
        predicted_representation = outputs.logits

        # Calculate dot product between projected and predicted representations
        dot_product = torch.matmul(projected_representation, predicted_representation.t())

        # Compute softmax along rows
        softmax_output = torch.nn.functional.softmax(dot_product, dim=1)

        # Calculate loss using negative log likelihood
        loss = -torch.mean(torch.log(torch.diag(softmax_output) + 1e-4))

        return loss

# Define subclassed Trainer
class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        loss = InfoNCELoss()(outputs, labels)
        return (loss, outputs) if return_outputs else loss



# Define compute_metrics function
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    accuracy = accuracy_score(labels, preds)
    precision = precision_score(labels, preds)
    recall = recall_score(labels, preds)
    f1 = f1_score(labels, preds)
    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1
    }


# Define Trainer arguments
args = TrainingArguments(
    output_dir="output",
    num_train_epochs=1,
    per_device_train_batch_size=8
)

# Use the custom Trainer
trainer = CustomTrainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)

# Train the model
trainer.train()
trainer.evaluate()

Cross Entropy loss

In [ ]:
import pandas as pd
import json

with open('./annotated_data-train.json', 'r') as f:
    data = json.load(f)


df1 = pd.DataFrame(data)

with open('./annotated_data-test.json', 'r') as f:
    data = json.load(f)

df2 = pd.DataFrame(data)

df=pd.concat([df1,df2])

df=df[['text','label']]

label_map = {'noHate': 0, 'hate': 1}

# Use the map function to apply the mapping
df['label_value'] = df['label'].map(label_map)

df.reset_index(drop=True, inplace=True)

data = df[['text','label_value']]

import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
import torch
from transformers import TrainingArguments, Trainer
from transformers import BertTokenizer, BertForSequenceClassification

from transformers import BertTokenizer, BertForSequenceClassification
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased',num_labels=2)

model = model.to('cuda')

X = list(data["text"])
y = list(data["label_value"])
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2,stratify=y)
X_train_tokenized = tokenizer(X_train, padding=True, truncation=True, max_length=512)
X_val_tokenized = tokenizer(X_val, padding=True, truncation=True, max_length=512)

X_train_tokenized.keys()

class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels=None):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        if self.labels:
            item["labels"] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.encodings["input_ids"])


train_dataset = Dataset(X_train_tokenized, y_train)
val_dataset = Dataset(X_val_tokenized, y_val)



def compute_metrics(p):
    print(type(p))
    pred, labels = p
    pred = np.argmax(pred, axis=1)

    accuracy = accuracy_score(y_true=labels, y_pred=pred)
    recall = recall_score(y_true=labels, y_pred=pred)
    precision = precision_score(y_true=labels, y_pred=pred)
    f1 = f1_score(y_true=labels, y_pred=pred)

    return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1}

# Define Trainer

from transformers import Trainer
from transformers import TrainingArguments
args = TrainingArguments(
    output_dir="output",
    num_train_epochs=1,
    per_device_train_batch_size=8

)
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)

trainer.train()

trainer.evaluate()

Triplet Loss

In [ ]:
import pandas as pd
import json
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from transformers import TrainingArguments, Trainer, BertTokenizer, BertForSequenceClassification
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score

# Load data from JSON files
with open('./annotated_data-train.json', 'r') as f:
    train_data = json.load(f)

with open('./annotated_data-test.json', 'r') as f:
    test_data = json.load(f)

# Convert data to Pandas DataFrame
train_df = pd.DataFrame(train_data)
test_df = pd.DataFrame(test_data)

# Concatenate train and test dataframes
df = pd.concat([train_df, test_df])

# Select 'text' and 'label' columns
df = df[['text', 'label']]

# Map label values to integers
label_map = {'noHate': 0, 'hate': 1}
df['label_value'] = df['label'].map(label_map)

# Reset index
df.reset_index(drop=True, inplace=True)

# Select 'text' and 'label_value' columns
data = df[['text', 'label_value']]

# Initialize BERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

# Move model to GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Split data into train and validation sets
X = list(data["text"])
y = list(data["label_value"])
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1, stratify=y)

# Tokenize train and validation sets
X_train_tokenized = tokenizer(X_train, padding=True, truncation=True, max_length=512)
X_val_tokenized = tokenizer(X_val, padding=True, truncation=True, max_length=512)

# Define Dataset class
class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

# Create train and validation datasets
train_dataset = Dataset(X_train_tokenized, y_train)
val_dataset = Dataset(X_val_tokenized, y_val)

# Define triplet loss function
class TripletLoss(nn.Module):
    def __init__(self, margin=0.01):
        super(TripletLoss, self).__init__()
        self.margin = margin

    def forward(self, anchor, positive, negative):
        # Ensure the shapes of anchor, positive, and negative tensors are compatible
        anchor = anchor.view(anchor.size(0), -1)
        positive = positive.view(positive.size(0), -1)
        negative = negative.view(negative.size(0), -1)

        # Calculate pairwise distances
        distance_positive = torch.norm(anchor - positive, p=2, dim=1)
        distance_negative = torch.norm(anchor - negative, p=2, dim=1)

        # Compute loss
        loss = F.relu(distance_positive - distance_negative + self.margin)
        return loss.mean()

# Define subclassed Trainer
class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        logits = outputs.logits
        loss = TripletLoss()(logits, labels, labels)  # Using labels as placeholders for anchor, positive, and negative
        return (loss, outputs) if return_outputs else loss

# Define compute_metrics function
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    accuracy = accuracy_score(labels, preds)
    precision = precision_score(labels, preds)
    recall = recall_score(labels, preds)
    f1 = f1_score(labels, preds)
    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1
    }

# Define Trainer arguments
args = TrainingArguments(
    output_dir="output",
    num_train_epochs=1,
    per_device_train_batch_size=16
)

# Use the custom Trainer
trainer = CustomTrainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)

# Train the model
trainer.train()
trainer.evaluate()


N-pair loss

In [ ]:
import pandas as pd
import json
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from transformers import TrainingArguments, Trainer, BertTokenizer, BertForSequenceClassification
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score

# Load data from JSON files
with open('./annotated_data-train.json', 'r') as f:
    train_data = json.load(f)

with open('./annotated_data-test.json', 'r') as f:
    test_data = json.load(f)

# Convert data to Pandas DataFrame
train_df = pd.DataFrame(train_data)
test_df = pd.DataFrame(test_data)

# Concatenate train and test dataframes
df = pd.concat([train_df, test_df])

# Select 'text' and 'label' columns
df = df[['text', 'label']]

# Map label values to integers
label_map = {'noHate': 0, 'hate': 1}
df['label_value'] = df['label'].map(label_map)

# Reset index
df.reset_index(drop=True, inplace=True)

# Select 'text' and 'label_value' columns
data = df[['text', 'label_value']]

# Initialize BERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

# Move model to GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Split data into train and validation sets
X = list(data["text"])
y = list(data["label_value"])
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1, stratify=y)

# Tokenize train and validation sets
X_train_tokenized = tokenizer(X_train, padding=True, truncation=True, max_length=512)
X_val_tokenized = tokenizer(X_val, padding=True, truncation=True, max_length=512)

# Define Dataset class
class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

# Create train and validation datasets
train_dataset = Dataset(X_train_tokenized, y_train)
val_dataset = Dataset(X_val_tokenized, y_val)

# Define N-pair loss function
class NpairLoss(nn.Module):
    def __init__(self, l2_reg=0.02):
        super(NpairLoss, self).__init__()
        self.l2_reg = l2_reg

    def forward(self, anchor, positive, target):
        batch_size = anchor.size(0)
        target = target.view(target.size(0), 1)

        target = (target == torch.transpose(target, 0, 1)).float()
        target = target / torch.sum(target, dim=1, keepdim=True).float()

        logit = torch.matmul(anchor, torch.transpose(positive, 0, 1))
        loss_ce = F.cross_entropy(logit, target)
        l2_loss = torch.sum(anchor**2) / batch_size + torch.sum(positive**2) / batch_size

        loss = loss_ce + self.l2_reg*l2_loss*0.25
        return loss

# Define subclassed Trainer
class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        logits = outputs.logits
        loss = NpairLoss()(logits, logits, labels)  # Using logits as anchor and positive
        return (loss, outputs) if return_outputs else loss

# Define compute_metrics function
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    accuracy = accuracy_score(labels, preds)
    precision = precision_score(labels, preds)
    recall = recall_score(labels, preds)
    f1 = f1_score(labels, preds)
    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1
    }

# Define Trainer arguments
args = TrainingArguments(
    output_dir="output",
    num_train_epochs=1,
    per_device_train_batch_size=16
)

# Use the custom Trainer
trainer = CustomTrainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)

# Train the model
trainer.train()
trainer.evaluate()


ROBERTa

InfoNCE

In [ ]:
import pandas as pd
import json
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
import torch
from transformers import TrainingArguments, Trainer
from transformers import RobertaTokenizer, RobertaForSequenceClassification
from torch.nn import CrossEntropyLoss
import torch.nn.functional as F

# Load data from JSON files
with open('./annotated_data-train.json', 'r') as f:
    train_data = json.load(f)

with open('./annotated_data-test.json', 'r') as f:
    test_data = json.load(f)

# Convert data to Pandas DataFrame
train_df = pd.DataFrame(train_data)
test_df = pd.DataFrame(test_data)

# Concatenate train and test dataframes
df = pd.concat([train_df, test_df])

# Select 'text' and 'label' columns
df = df[['text', 'label']]

# Map label values to integers
label_map = {'noHate': 0, 'hate': 1}
df['label_value'] = df['label'].map(label_map)

# Reset index
df.reset_index(drop=True, inplace=True)

# Select 'text' and 'label_value' columns
data = df[['text', 'label_value']]

# Initialize RoBERTa tokenizer and model
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
model = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=2)

# Move model to GPU
model = model.to('cuda')

# Split data into train and validation sets
X = list(data["text"])
y = list(data["label_value"])
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, stratify=y)

# Tokenize train and validation sets
X_train_tokenized = tokenizer(X_train, padding=True, truncation=True, max_length=512)
X_val_tokenized = tokenizer(X_val, padding=True, truncation=True, max_length=512)

# Define Dataset class
class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

# Create train and validation datasets
train_dataset = Dataset(X_train_tokenized, y_train)
val_dataset = Dataset(X_val_tokenized, y_val)

# Define InfoNCE loss function
class InfoNCELoss(torch.nn.Module):
    def __init__(self):
        super(InfoNCELoss, self).__init__()

    def forward(self, outputs, labels):
        # Retrieve the projected and predicted representations
        projected_representation = outputs.logits
        predicted_representation = outputs.logits

        # Calculate dot product between projected and predicted representations
        dot_product = torch.matmul(projected_representation, predicted_representation.t())

        # Compute softmax along rows
        softmax_output = torch.nn.functional.softmax(dot_product, dim=1)

        # Calculate loss using negative log likelihood
        loss = -torch.mean(torch.log(torch.diag(softmax_output) + 1e-4))

        return loss

# Define subclassed Trainer
class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        loss = InfoNCELoss()(outputs, labels)
        return (loss, outputs) if return_outputs else loss

# Define compute_metrics function
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    accuracy = accuracy_score(labels, preds)
    precision = precision_score(labels, preds)
    recall = recall_score(labels, preds)
    f1 = f1_score(labels, preds)
    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1
    }

# Define Trainer arguments
args = TrainingArguments(
    output_dir="output",
    num_train_epochs=1,
    per_device_train_batch_size=8
)

# Use the custom Trainer
trainer = CustomTrainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)

# Train the model
trainer.train()

# Evaluate the model
trainer.evaluate()


Cross Entropy

In [ ]:
import pandas as pd
import json
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
import torch
from transformers import TrainingArguments, Trainer
from transformers import RobertaTokenizer, RobertaForSequenceClassification

# Load data from JSON files
with open('./annotated_data-train.json', 'r') as f:
    train_data = json.load(f)

with open('./annotated_data-test.json', 'r') as f:
    test_data = json.load(f)

# Concatenate train and test dataframes
df = pd.concat([pd.DataFrame(train_data), pd.DataFrame(test_data)])

# Select 'text' and 'label' columns
df = df[['text', 'label']]

# Map label values to integers
label_map = {'noHate': 0, 'hate': 1}
df['label_value'] = df['label'].map(label_map)

# Reset index
df.reset_index(drop=True, inplace=True)

data = df[['text', 'label_value']]

# Initialize RoBERTa tokenizer and model
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
model = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=2)

# Move model to GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Split data into train and validation sets
X = list(data["text"])
y = list(data["label_value"])
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, stratify=y)

# Tokenize train and validation sets
X_train_tokenized = tokenizer(X_train, padding=True, truncation=True, max_length=512)
X_val_tokenized = tokenizer(X_val, padding=True, truncation=True, max_length=512)

# Define Dataset class
class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels=None):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        if self.labels:
            item["labels"] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.encodings["input_ids"])

# Create train and validation datasets
train_dataset = Dataset(X_train_tokenized, y_train)
val_dataset = Dataset(X_val_tokenized, y_val)

# Define compute_metrics function
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    accuracy = accuracy_score(labels, preds)
    precision = precision_score(labels, preds)
    recall = recall_score(labels, preds)
    f1 = f1_score(labels, preds)
    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1
    }

# Define Trainer arguments
args = TrainingArguments(
    output_dir="output",
    num_train_epochs=1,
    per_device_train_batch_size=8
)

# Use the Trainer
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)

# Train the model
trainer.train()

# Evaluate the model
trainer.evaluate()


Triplet loss

In [ ]:
import pandas as pd
import json
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from transformers import TrainingArguments, Trainer, RobertaTokenizer, RobertaForSequenceClassification
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score

# Load data from JSON files
with open('./annotated_data-train.json', 'r') as f:
    train_data = json.load(f)

with open('./annotated_data-test.json', 'r') as f:
    test_data = json.load(f)

# Convert data to Pandas DataFrame
train_df = pd.DataFrame(train_data)
test_df = pd.DataFrame(test_data)

# Concatenate train and test dataframes
df = pd.concat([train_df, test_df])

# Select 'text' and 'label' columns
df = df[['text', 'label']]

# Map label values to integers
label_map = {'noHate': 0, 'hate': 1}
df['label_value'] = df['label'].map(label_map)

# Reset index
df.reset_index(drop=True, inplace=True)

# Select 'text' and 'label_value' columns
data = df[['text', 'label_value']]

# Initialize RoBERTa tokenizer and model
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
model = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=2)

# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Split data into train and validation sets
X = list(data["text"])
y = list(data["label_value"])
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1, stratify=y)

# Tokenize train and validation sets
X_train_tokenized = tokenizer(X_train, padding=True, truncation=True, max_length=512)
X_val_tokenized = tokenizer(X_val, padding=True, truncation=True, max_length=512)

# Define Dataset class
class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

# Create train and validation datasets
train_dataset = Dataset(X_train_tokenized, y_train)
val_dataset = Dataset(X_val_tokenized, y_val)

# Define triplet loss function
class TripletLoss(nn.Module):
    def __init__(self, margin=0.01):
        super(TripletLoss, self).__init__()
        self.margin = margin

    def forward(self, anchor, positive, negative):
        # Ensure the shapes of anchor, positive, and negative tensors are compatible
        anchor = anchor.view(anchor.size(0), -1)
        positive = positive.view(positive.size(0), -1)
        negative = negative.view(negative.size(0), -1)

        # Calculate pairwise distances
        distance_positive = torch.norm(anchor - positive, p=2, dim=1)
        distance_negative = torch.norm(anchor - negative, p=2, dim=1)

        # Compute loss
        loss = F.relu(distance_positive - distance_negative + self.margin)
        return loss.mean()

# Define subclassed Trainer
class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        logits = outputs.logits
        loss = TripletLoss()(logits, labels, labels)  # Using labels as placeholders for anchor, positive, and negative
        return (loss, outputs) if return_outputs else loss

# Define compute_metrics function
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    accuracy = accuracy_score(labels, preds)
    precision = precision_score(labels, preds)
    recall = recall_score(labels, preds)
    f1 = f1_score(labels, preds)
    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1
    }

# Define Trainer arguments
args = TrainingArguments(
    output_dir="output",
    num_train_epochs=1,
    per_device_train_batch_size=16
)

# Use the custom Trainer
trainer = CustomTrainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)

# Train the model
trainer.train()
trainer.evaluate()


N-PAIR loss

In [ ]:
import pandas as pd
import json
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from transformers import TrainingArguments, Trainer, RobertaTokenizer, RobertaForSequenceClassification
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score

# Load data from JSON files
with open('./annotated_data-train.json', 'r') as f:
    train_data = json.load(f)

with open('./annotated_data-test.json', 'r') as f:
    test_data = json.load(f)

# Convert data to Pandas DataFrame
train_df = pd.DataFrame(train_data)
test_df = pd.DataFrame(test_data)

# Concatenate train and test dataframes
df = pd.concat([train_df, test_df])

# Select 'text' and 'label' columns
df = df[['text', 'label']]

# Map label values to integers
label_map = {'noHate': 0, 'hate': 1}
df['label_value'] = df['label'].map(label_map)

# Reset index
df.reset_index(drop=True, inplace=True)

# Select 'text' and 'label_value' columns
data = df[['text', 'label_value']]

# Initialize RoBERTa tokenizer and model
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
model = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=2)

# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Split data into train and validation sets
X = list(data["text"])
y = list(data["label_value"])
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1, stratify=y)

# Tokenize train and validation sets
X_train_tokenized = tokenizer(X_train, padding=True, truncation=True, max_length=512)
X_val_tokenized = tokenizer(X_val, padding=True, truncation=True, max_length=512)

# Define Dataset class
class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

# Create train and validation datasets
train_dataset = Dataset(X_train_tokenized, y_train)
val_dataset = Dataset(X_val_tokenized, y_val)

# Define N-pair loss function
class NpairLoss(nn.Module):
    def __init__(self, l2_reg=0.02):
        super(NpairLoss, self).__init__()
        self.l2_reg = l2_reg

    def forward(self, anchor, positive, target):
        batch_size = anchor.size(0)
        target = target.view(target.size(0), 1)

        target = (target == torch.transpose(target, 0, 1)).float()
        target = target / torch.sum(target, dim=1, keepdim=True).float()

        logit = torch.matmul(anchor, torch.transpose(positive, 0, 1))
        loss_ce = F.cross_entropy(logit, target)
        l2_loss = torch.sum(anchor**2) / batch_size + torch.sum(positive**2) / batch_size

        loss = loss_ce + self.l2_reg*l2_loss*0.25
        return loss

# Define subclassed Trainer
class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        logits = outputs.logits
        loss = NpairLoss()(logits, logits, labels)  # Using logits as anchor and positive
        return (loss, outputs) if return_outputs else loss

# Define compute_metrics function
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    accuracy = accuracy_score(labels, preds)
    precision = precision_score(labels, preds)
    recall = recall_score(labels, preds)
    f1 = f1_score(labels, preds)
    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1
    }

# Define Trainer arguments
args = TrainingArguments(
    output_dir="output",
    num_train_epochs=1,
    per_device_train_batch_size=16
)

# Use the custom Trainer
trainer = CustomTrainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)

# Train the model
trainer.train()
trainer.evaluate()


Code to extract representations

In [ ]:
# Extract representations for train dataset
train_representations = []
with torch.no_grad():
    for batch in torch.utils.data.DataLoader(train_dataset, batch_size=8):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        outputs = model(input_ids, attention_mask=attention_mask, output_hidden_states=True)  # Ensure hidden states are returned
        last_hidden_states = outputs.hidden_states[-1]  # Access last layer's hidden states
        train_representations.append(last_hidden_states.cpu().numpy())

# Extract representations for validation dataset
val_representations = []
with torch.no_grad():
    for batch in torch.utils.data.DataLoader(val_dataset, batch_size=8):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        outputs = model(input_ids, attention_mask=attention_mask, output_hidden_states=True)  # Ensure hidden states are returned
        last_hidden_states = outputs.hidden_states[-1]  # Access last layer's hidden states
        val_representations.append(last_hidden_states.cpu().numpy())

# Convert representations to numpy arrays
train_representations = np.concatenate(train_representations, axis=0)
val_representations = np.concatenate(val_representations, axis=0)

